---
>闇の夜道に松明\
>蓋し、灯されしもののあないみじや
---

# PyTorch

PyTorch は科学技術計算向けパッケージであり、
- GPUを使った高速計算が可能
  - 普通やらないがNumPyの代わりに使える
  - NumPyをGPU付きで使いたいならCuPyがある\
    PyTorchによりも先に登場したPFNのChainerはCuPyを使う
- 柔軟かつ高速にディープラーニングプラットフォームを構築可能
である。

https://pytorch.org/ にほぼすべての情報がある。

英語ではあるが、関連するドキュメントが十分かつ分かりやすく準備されている

PyTorchの以前のロゴは次の通り

<img src="http://class.west.sd.keio.ac.jp/dataai/text/pytorchlogo.jpeg" width="50%">

リングフィットアドベンチャーに似ているので変えたとか変えないとか

<img src="http://class.west.sd.keio.ac.jp/dataai/text/rfa.png" width="20%">

## PyTorchで線形回帰

PyTorchを用いても線形回帰（Linear Regression）を実現できる

ここでは、Deep Learning実装で用いるのが主たる目的であるPyTorchを使って、線形回帰を実装する

- データとして、手入力の配列データで実装する

- ハイパーパラメータについても、最初に宣言しておくので、値を変えて様々トライしてみるとよいであろう

# PyTorchの実装例

PyTorchについて学んでいくが、まずはPyTorchを用いた機械学習コードのその実行イメージを掴む
- コードの各行の意味にこだわらず、全体の雰囲気を感じ取ること
  - 英語も文法から入るとつまらないし、自由に使いこなすという観点からは良い方法といえない
  - 使うことを念頭に学ぶのであれば、英語を聞いて話すのと同様、細かいことにこだわらず、コードを見て書けばよい
- 次回のノートブックで基本的なテンソルの扱い方などを学ぶが、そういう内容を深く知らなくてもなんとなくできてしまうという感覚を身につける
  - 自分が何を学んでいるのか、何を学ばないといけないのかを知るには、適度にシンプルな最終形を示すのが一番


In [ ]:
import torch            
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
input_size = 1
output_size = 1
num_epochs = 1000
learning_rate = 0.002

手入力で適当なデータセットを準備する

- 既に用いたことのあるデータを利用してもよい

 `nn.Linear` に対する入力は `(N,∗,in_features)` であるため `reshape` が必要となる

- `N`は次元、`*`には任意の次元を追加できるが今回は1次元データであるため、特に指定しない

表示して内容を確認する

- 再びのPyTorchでmodelに食わせるためのreshapeであるが、`reshape(15, 1)`と同じこと

In [ ]:
x_train = np.array([3.3, 4.4, 5.5, 6.71, 6.93, 4.168, 9.779, 6.182, 7.59, 2.167,
                    7.042, 10.791, 5.313, 7.997, 3.1], dtype=np.float32)

y_train = np.array([1.7, 2.76, 2.09, 3.19, 1.694, 1.573, 3.366, 2.596, 2.53, 1.221,
                    2.827, 3.465, 1.65, 2.904, 1.3], dtype=np.float32)
print("ORG:", x_train)
x_train = x_train.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)
print("RESHAPE:",x_train)

ここでは、おおよその記述スタイルを学ぶという観点から、詳細には触れず要点のみ触れる

実装は、まずデータセットを準備し、次にモデルを構成する

- データセットの準備(既に終了している)
- モデルの定義
- ロス関数の定義
- 最適化手法の定義
- 学習
  - 順伝搬で出力を計算(内部でforwardを呼び出す)
  - 出力値と正解値から誤差を計算
    - ロス関数の利用、内部でbackwordを呼び出すが計算式はmodelのforwardを求める際に自動で獲得されており、outputに含まれている
  - 重みによる誤差の偏微分値を計算
  - 誤差を逆伝搬
    - 最適化手法でstepを呼び出す

ネットワークの構成は次の通り

-  `nn.Module` を継承したクラスを作成  
クラスを継承して作成するのがPyTorchの設計の基本スタイルの一つ
- `__init__()` に層オブジェクトを定義  
コンストラクタを用いて、層のオブジェクトを定義
- `forward()` に順方向の処理

In [ ]:
class LinearRegression(nn.Module):
  def __init__(self, input_size, output_size):
    super(LinearRegression, self).__init__()
    self.linear = nn.Linear(input_size, output_size)
  def forward(self, x):
    out = self.linear(x)
    return out
model = LinearRegression(input_size, output_size)

Loss計算とOptimizerについて

- Loss計算について、線形回帰であるため、誤差は平均二乗誤差（mean squared error）を用いる
- Optimizerについて、ここではシンプルなSGD（Stochastic Gradient Descent）を指定する


In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

学習について

- PyTorchでは、学習部分の繰り返し、すなわちエポックの記述をforで実装するのが一般的
- 各エポックで、勾配のクリアを忘れずに行う
  - 実際には、zero_grad()を用いてクリアする
- パラメータは、optimizer.step()メソッドを用いて更新する
- ここでは、100エポック毎のlossを表示する
  - 一般的なif文の記述スタイルを用いている
- 最後にモデルを保存する
  - モデルの保存は、torch.saveを用いる
  - 学習済みモデルをmodel.state_dict()メソッドで取り出す
  - この学習済みモデルを保存し再利用することで、学習させずに学習結果を利用したアプリケーションが設計できる



In [ ]:
for epoch in range(num_epochs):
    inputs = torch.tensor(x_train) # 新しい書き方に修正しています
    targets = torch.tensor(y_train)
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 100 == 0:
        print('Epoch [%d/%d], Loss: %.4f' % (epoch + 1, num_epochs, loss.item()))
# save the model
torch.save(model.state_dict(), 'model.pkl')

評価する

- 予測結果と元のデータとを比較する
- 勾配(grad)の情報を保有するTensorはそのままnumpy arrayに変換できないため、detach()する

In [ ]:
predicted = model(torch.tensor(x_train)).detach().numpy()
plt.plot(x_train, y_train, 'ro', label='Original data')
plt.plot(x_train, predicted, label='Fitted line')
plt.legend()
plt.show()

dictでモデルが取り出せるが、モデルのパラメタを別途直接指定したい、個別に取り出したい等の場合は、`torch.nn.Parameter`オブジェクトを扱う

- `torch.nn.Parameter`クラスの`__init__`関数にはtorchテンソルを指定することで、全体を設定できる

- 個別には次のようにする
  - `＜モデル名＞.＜レイヤー名＞.weight`プロパティに重みが指定できる
  - `＜モデル名＞.＜レイヤー名＞.baias`プロパティにバイアスが指定できる

- 既に述べた通り重みやバイアスといったパラメーターなどの`torch.nn.Module`全体の状態は、`＜モデル名＞.state_dict()`メソッドで取得できる
  - パラメーターを最適化で使う際は専用の取得方法があり、`＜モデル名＞.parameters()`メソッドで取得する

In [ ]:
list(model.parameters())

## PyTorchでNNによる分類

### 学習データ
scikit-learnに含まれるワイン分類データセットを用いて、PyTorchでワインの等級分けを行う

- このデータセットには、13個の特徴量が含まれている
- 出力は本来class_0, 1, 2の3種類であるが、今回はclass_0,1のみを判別するため2つ用いる
- その他は次のようなモデルを想定する
  - 入力層(x)は13個:（特徴量に等しい）
  - 隠れ層(fc1): 全結合
  - 出力層(fc2): 全結合
  - softmax層: 判定するラベル数と同数の2ノード
    - 確率が足して1になるように調整する
 - 出力 ラベル0と1の確率

このネットワーク図は次の通り
- 記述とこの図の対応がとれるように

<img src="http://class.west.sd.keio.ac.jp/dataai/text/model1.png" width=500>

復習として、Softmax関数は、$d$次元のベクトル${\bf y} \in \mathbb{R}^d$が与えられたとき、各次元の値の合計が1になるように正規化する

- すなわち、確率分布のような出力を任意の実数ベクトルから作ることができる

- ${\bf y}$の$i$番目の次元を$y_i$と書くと，Softmax関数は

$$
p_i = \frac{e^{y_i}}{\sum_{j=1}^d e^{y_j}}
$$

と表せる

### 学習

学習の流れは次の通り

- 順伝搬で出力を計算(図中①)
- 出力値と正解ラベルから誤差を計算(図中②)
- 重みによる誤差の偏微分値を計算(図中③)
- 誤差を逆伝搬(図中④)
- 更新した重みで500エポック計算

実装は、まずデータセットを準備し、次にネットワークを構成する

### 前準備
`import torch`としてtorchを利用

基本これだけでよいが、プログラムの記述上省略したほうが簡潔でよい、わかりやすいといった観点から、下記は必須ではないが、定義して省略形で使えるように準備する場合もある
- `import torch.nn as nn`: nnでニューラルネットワーク関数を参照
- `import tourch.nn.functional as F`： Fで活性化関数定義
- `import torch.optim as optim` ：optimで最適化関数定義
- `from torch.utils.data import DataLoader, TensorDataset` ：データローダ、データセット利用を簡略化

簡単にpython文法を復習する
- `import A.B`とすると、Aの中のBという機能を利用できる
  - この場合、使うたびにA.Bと名前を指定する
  - A全体を読み込むより、A.Bしか読み込まないためメモリ利用効率は向上するが、利用の便宜上は何も変わらない
  - そこで、`from A import B`とすると、読み込みもA.BのみでBとして利用できる
- as はエイリアス(別名定義)である
  - `import A.B as C`とすれば、Cという名前で当該機能を利用できる
- 両方まとめて`from A.B import C.D as E`とできる

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

scikit-learnをデータセット入手だけに利用する
- ワインのデータセット、およびデータセットから教師データとテストデータを分離する関数もscikit-learn提供の機能を利用する

In [ ]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

pandasも利用する

In [ ]:
import pandas as pd

### 学習データの準備
####  ワインデータセットの読み込み
load_wineを用いてデータセットを読み込む

データの中身を煩雑だが確認する
- 13個の特徴点リストを含むデータセットである
- feature_namesは、13個の特徴点の名称を含むデータセットである
- targetは、dataそれぞれがどのワイン種別に属するかを表している正解ラベルである
- target_namesはワインのラベルで、class_0、 class_1といった等級分けである


In [ ]:
wine = load_wine()
wine

pandasのDataFrameを用いて、feature_namesをラベルとしてデータを表示する
- 中身をよりわかりやすく確認できる

In [ ]:
pd.DataFrame(wine.data, columns=wine.feature_names)

次に、2のラベルがついているデータを省いて、0と1だけにする

今回利用するデータが、たまたまクラスが0, 1, 2の順に並んでいるため、下記のようにすれば十分であるが、これでは汎用性がない

In [ ]:
wine_data2 = wine.data[0:130]
wine_target2 = wine.target[0:130]

今後のことも考え、pandasのdropを用い、より汎用性の高い方法で分離する

`pandas.DataFrame.drop(labels=None,axis=0,index=None,columns=None,level=None,inplace=False,errors=’raise’)`

とする

各パラメータは次の通り

- labels: ラベル名またはラベル名のリスト
  -  (省略可能)初期値None  
  -  消去したい列データor行データのラベルを指定
- axis: 0または`index`,1または`columns`
  -  (省略可能)初期値0  
  -  行データ(0または`index`)を削除するか列データ(1または`columns`)を削除するかを指定
- index,columns: ラベル名またはラベル名のリスト
  - (省略可能)初期値None  
  - 省略したいラベル名を行データ(index)、列データ(columns)で個別に指定
- level: intもしくは階層名
  -  (省略可能)初期値None  
  -  ラベルとして利用する階層を指定
- inplace: bool値
  - (省略可能)初期値False  
  - Trueでvoidとなり元データに変更を反映
- errors:	`ignore`または`raise`
  - (省略可能)初期値’raise’  
  - ‘ignore’でエラーを無視し処理続行

教師データ、正解ラベルの両方からclass2のデータを消す必要があるため、一度データをマージして削除する

ここでは、横方向に連結して、第0列が正解ラベル、第1列以降に特徴量データが並ぶようにデータセットを構成する

- データの連結には`pd.concat([df1, df4], axis=1)`とする
  - 横方向に連結するため axis=1 を指定
  - このとき紐付けは 連結方向でないラベル=index について行われる
  - 連結方向のラベルにあたる columns はそのまま維持される

pd.concat([wine.data,wine.target], axis=1)としたいところだが、型が合わないといわれるので変換する
- 暗黙でやれといいたい
- ラベルが勝手につくと厄介なので、ここでラベルを付けておく

In [ ]:
wine_cat = pd.concat([pd.DataFrame(wine.data, columns=wine.feature_names),
                      pd.DataFrame(wine.target, columns=['class'])], axis=1)
wine_cat.head(5)

結合できたので、ここから、クラス2のデータを削除する

消す前に、練習として、classが2であるデータだけ抽出するには、次のようにする

In [ ]:
wine_cat[wine_cat['class'] == 2].head(5)

あとで削除したときにどれだけ削除したかわかるように、データのサイズも確認しておく

In [ ]:
wine_cat.shape

実際にデータを削除する
- これには、dropメソッドを使う
- 先に示したように、データはインデックスを使って選択が可能で、dropは該当したデータだけ削除する
- inplace=Trueとして、結果を易直接wine_catに代入する

In [ ]:
wine_cat[wine_cat['class']==2].index

In [ ]:
wine_cat.drop(wine_cat[wine_cat['class'] == 2].index, inplace=True)

行を抽出するのではなく列を抽出するため、`[:,:13]`となる
- 13であるが、これは0から12を意味する
  - 0から13個の情報、未満と解釈してもよい

In [ ]:
wine_data = wine_cat.values[:,:13]
wine_target = wine_cat.values[:,13]
print(wine_data, len(wine_data))
print(wine_target, len(wine_target))

### データセットの分割

データセットから取り出した特徴量XおよびラベルYについて、さらに「トレーニング用」と「テスト用」のデータに分割する


In [ ]:
Train_X, Test_X, Train_Y, Test_Y = train_test_split(wine_data, wine_target, test_size=0.25)

内容を確認する130$\times$0.25$\approx$33である。

In [ ]:
Train_X.shape, Test_X.shape

### PyTorchテンソルへの変換と訓練データセット作成

from_numpyを使って変換(こちらはよくありますが古い書き方です)

In [ ]:
train_X = torch.from_numpy(Train_X).float()
train_Y = torch.from_numpy(Train_Y).long()
test_X = torch.from_numpy(Test_X).float()
test_Y = torch.from_numpy(Test_Y).long()

torch.tensorを使って変換

In [ ]:
train_X = torch.tensor(Train_X, dtype=torch.float)
train_Y = torch.tensor(Train_Y, dtype=torch.long)
test_X = torch.tensor(Test_X, dtype=torch.float)
test_Y = torch.tensor(Test_Y, dtype=torch.long)

今風かつ推奨の書き方

In [ ]:
train_X = torch.FloatTensor(Train_X)
train_Y = torch.LongTensor(Train_Y)
test_X = torch.FloatTensor(Test_X)
test_Y = torch.LongTensor(Test_Y)

中身を確認

In [ ]:
train_X.shape, test_X.shape, train_Y.shape, test_Y.shape

学習には、TensorDatasetを用いて一つのテンソルの中に入れる必要がある

In [ ]:
train = TensorDataset(train_X, train_Y)

中身を確認する

In [ ]:
train[0], len(train)

ちなみに、`train.view()`は失敗する
- このtrainは、ミニバッチ専用の型を持ち、テンソル型ではない
  - 以前のバージョンでは見えた気がするので、今後見えるかもしれない

In [ ]:
type(train)

訓練データセットからミニバッチで順にデータを学習する
- ミニバッチではデータをミニバッチサイズ分まとめて並列的に学習させるため、入力させるデータの次元が一つ増えてミニバッチサイズ分束ねたテンソルとなる
  - PyTorchでは、入力される方(モデル)も対応して束ねたテンソルを受け付けることができるようになる
  - 自動化されているのでサイズなどは気にせずともよいが、込み入ると気にしないといけない場合もある
- ここでは、バッチサイズを15とする
  - かなり小さく、モデルにもよるがGPUなら100を超えてもよい
- shuffle=Trueとしてでたらめに並び替える
  - shuffleは、取り出すたびにランダムに取り出すのではない
  - まずデータをシャッフルして、その後順に取り出す、全てが取り出し終わったらもう一度シャッフルしてデータを作り直す、という処理が行われる
  - 訓練データは普通Trueを指定

In [ ]:
train_loader = DataLoader(train, batch_size=15, shuffle=True)

### モデル定義

一番最初に述べたネットワークモデルを構築する。既に存在するクラスを継承して用いると簡単であるため、PyTorchでは通常継承して利用

- `__init__`は、インスタンス生成時に呼ばれるコンストラクタ(1)
- superは継承元の親クラスを意味し、その親クラスのコンストラクタを呼び出す(2)
- nn.Linierでは、全結合層を構成
  - 入力の数を13、中間層のノード数を128と指定する(3)
- さらに、128入力2出力の全結合層を構成(4)
- 次に、ネットワークにデータを通して出力値を求める関数を作成
  - 名前はforwardとする必要がある(5)
- fc1の活性化関数に例としてReLUを利用(6)
- その結果を次のfc2に投入(7)
- 値を返す(8)
  - 本来は最後にsoftmax関数を配置するが、次のノートブックで説明するようにPyTorchのCrossEntropyLoss処理はsoftmax処理も内包しているため、この場合softmax関数は不要
  - もしここでSoftmaxを指定する場合は、どの軸に対してsoftmaxを施すかを指定するためdim=0とする
    - 各自試すと良いが、誤ってsoftmaxを指定した場合でも、学習に失敗するといったことはないであろう
  - 結果を出力するため、戻り値をreturnする

In [ ]:
class Net(nn.Module):
  def __init__(self): #(1)
    super(Net, self).__init__() #(2)
    self.fc1 = nn.Linear(13, 128) #(3)
    self.fc2 = nn.Linear(128, 2) #(4)
  def forward(self, x): #(5)
    x = F.relu(self.fc1(x)) #(6)
    x = self.fc2(x) #(7)
    return x #(8)

インスタンス化してmodelを作成する
- このmodelにデータを投入する

In [ ]:
model = Net()

### 学習の実行

損失をどのように定義するかを決定する
- 今回は交差エントロピーを利用する
- また、確率的勾配効果法SGDを利用して最適値へと漸近させる

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

今回は500回エポックを廻す
- 損失を格納する変数を0で初期化しておく(1)
- ミニバッチからデータを順次取り出して、train_xとtrain_yに格納する(2)  
- 初期勾配を0にする(3)
- 図の①に相当するforward計算を行う。これには、modelにtrain_xを渡せばよい(4)
- 図の②に相当する損失計算を行う。すでに宣言したcriterionに、計算結果と教師データを渡す(5)
- 図の③に相当する処理として、ロスを後方に伝搬させる(6)
- 図の④に相当する処理として、その結果を用いてパラメータを更新する(7)
- 毎回のlossの値を積算する(8)
- 10回に一度結果を表示して確認する。epochは0スタートであることに注意する(9)

In [ ]:
for epoch in range(500):
  total_loss = 0 #(1)
  for train_x, train_y in train_loader: #(2)
    optimizer.zero_grad() #(3)
    output = model(train_x) #(4)
    loss = criterion(output, train_y) #(5)
    loss.backward() #(6)
    optimizer.step() #(7)
    total_loss += loss.data #(8)
  if(epoch+1)%10 == 0: #(9)
    print(epoch+1, total_loss)

### 精度の計算
まずは値を取り出して、testに対する答えを出す
- `.detach`は、含まれている勾配を計算する演算情報を削除することができる


In [ ]:
test_model = model(test_X).detach()

その結果と1の大きい方を結果とする
- test_yのデータと比較して、等しければ1、違っていれば0とし、全部足し合わせる
- その値を、データの総数で割り、平均を求める

In [ ]:
test_model

この行列は、それぞれのtestに対する評価値を保存している
- クラス0と考えられる値および、クラス1と考えられる値のセットである

値の大きい方が「推定した答え」であるので、こちらを選別する
- このように行列で値の大きい方をまとめて選別する特別な関数 torch.maxが準備されている

まずは、torch.maxについて簡単に説明する

#### torch.maxについて

モデルの出力はデータ数×クラス数の行列である。正解データはデータ数次元のベクトルである。これを比較するのが、torch.maxである。データ数xクラス数の行列の結果を変換する。

torch.maxは、行列（ベクトル）の最大値とそのインデックスを返す。以下の例で確認する。

|  |  |  |
|--|--|--|
|0.2|	1.4|	1.7|
|0.3|	1.5|	0.7|
|1.1|	1.3|	0.4|
|0.9|	2.3|	0.5|

このような例について、考えてみると、

縦で見れば、02, 0.3, 1.1, 0.9で最もスコアが高いのは1.1である。したがって、第2行が選択される。同様に、2, 3, 0が選択されるとわかる。下記のセルで答えが一致することを確認する。また、戻り値は配列の配列であり、値で取得するのか、行もしくは列番号で取得するのかによって、取得する場所を変える。

まずは、無理やり上記の行列を評価可能なテンソル型にする。この変換自体も、型がどのように扱われているかの理解を進めるであろう。

その後、torch.maxで最大を抽出する
- 行・列どちらで判断するかを0か1で指定する
- 0を指定すると列、1を指定すると行でみて最大値を取り出す

なお、類似の関数にtorch.argmaxも準備されている
- 違いはアルゴリズムで、argmaxの方が計算速度を速くすることができるが、同じデータが並んでいた場合、torch.maxは必ず最初の要素を返すのに対して、argmaxは最初の値を返す保証がなくなるという問題がある
- torch.argmaxは最大値を与えることはできず、最大であるindexのみ返す

In [ ]:
import numpy as np
testX = np.array([[0.2, 1.4, 1.7],[0.3, 1.5, 0.7],[1.1, 1.3, 0.4],[0.9, 2.3, 0.5]])
x = torch.tensor(testX, requires_grad=False, dtype=torch.float) # 新しい書き方に修正しています
print(x)
value, index = torch.max(x.data, 0) 
print("value0:", value, "index0:", index)
value, index = torch.max(x.data, 1)
print("value1:", value, "index1:", index)

さて、本題にもどって最大を抽出する

In [ ]:
torch.max(test_model, 1)

値が確認できたら、精度を求める

In [ ]:
result = torch.max(test_model, 1)[1]
accuracy = sum(test_Y.data.numpy() == result.numpy()) / len(test_Y.data.numpy())
accuracy

現在の設計ではこの程度
- あえてうまくいかないようにネットワークを小さくしている
- シンプルには学習パラメータを変更して精度を上げることができる

# 課題1(PyTorch NN改良)

同様に次の点を改良してみよう

**[改良1]** Learning Rate (lr)を$1 \over 10$の値に変更すると、accuracyの値は向上するかどうか、実際に確認しなさい
- `optim.SGD(model.parameters(), lr=0.01)`のlrの値を指す
- あくまでも結果は今回の例に限定されることに注意する
  
**[改良2]** 次のネットワークの改善の方で同様にパラメータを調整し、90%超えを狙いなさい

例えば、ネットワークを以下のように変更し、隠れ層をさらに深く、全結合層を5層に増やす

<img src="http://class.west.sd.keio.ac.jp/dataai/text/model2.png" width="70%">

以下、モデルの再定義例を示す
- 各自でaccuracyの値を確認しなさい

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(13, 128)
    self.fc2 = nn.Linear(128, 128)
    self.fc3 = nn.Linear(128, 128)
    self.fc4 = nn.Linear(128, 128)
    self.fc5 = nn.Linear(128, 128)
    self.fc6 = nn.Linear(128, 2)
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = F.relu(self.fc4(x))
    x = F.relu(self.fc5(x))
    x = self.fc6(x)
    return x

## PyTorchで手書き文字認識

シンプルなディープラーニングの例を示す

- 手書き文字認識といえば、MNISTが定番であるが、ここではさらに簡便なscikit-learnに付属する手書き文字の認識を行う
- 高々3層なのでディープといえるかどうかは微妙だが…

scikit-learnから、手書き数字の画像データを読み込んで表示する

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
digits_data = datasets.load_digits()
n_img = 10  # 表示する画像の数
plt.figure(figsize=(10, 4))
for i in range(n_img):
  ax = plt.subplot(2, 5, i+1)
  plt.imshow(digits_data.data[i].reshape(8, 8), cmap="Greys_r")
  ax.get_xaxis().set_visible(False)  # 軸を非表示に
  ax.get_yaxis().set_visible(False)
plt.show()
print("データの形状:", digits_data.data.shape)
print("ラベル:", digits_data.target[:n_img])

画像サイズが8×8と小さい、つまり解像度が低いため、つぶれた画像になっている
- より解像度の高い画像の認識については改めて学びます
  - scikit-learnが持っている画像データは利用しやすいように小さく構成されています
- 例として、0から9までの手書き数字の画像を表示したが、この程度の解像度しかない

このような手書き数字の画像が、scikit-learnの手書き文字データセットには1797枚含まれている
- 各画像は正解となる描かれた数字を表すラベルとセットになっている


scikit-learnのtrain_test_splitを使って、データを訓練用とテストに分割する
- 本来はこの方法ではないが、最初なので無理やり使ってみる
- 真似しないように

In [ ]:
import torch
from sklearn.model_selection import train_test_split
digit_images = digits_data.data
labels = digits_data.target
x_train, x_test, t_train, t_test = train_test_split(digit_images, labels)  # 25%がテスト用
# PyTorchで扱うためTensorに変換
x_train = torch.tensor(x_train, dtype=torch.float32)
t_train = torch.tensor(t_train, dtype=torch.int64) 
x_test = torch.tensor(x_test, dtype=torch.float32)
t_test = torch.tensor(t_test, dtype=torch.int64) 

`nn`モジュールの`Sequential`クラスによりモデルを構築する
- 今回は、あまり精度の良いモデルではないが、このサイズのデータセットであれば十分といえる
- 低解像度の画像は、ある意味畳み込み層やプーリング層を介した結果の画像ともいえなくもない

In [ ]:
from torch import nn
net = nn.Sequential(
  nn.Linear(64, 32),  # 全結合層
  nn.ReLU(),          # ReLU
  nn.Linear(32, 16),
  nn.ReLU(),
  nn.Linear(16, 10)   #0から9の10種の数字に対応
)
print(net)

学習として、モデルを訓練する

- 損失関数に交差エントロピー誤差を用い、最適化アルゴリズムにSGD(確率的勾配降下法)を用いる
- 引数に構築したネットワーク(net)のパラメータを渡す
- 順伝播は訓練データ、テストデータ両者で行い誤差を計算する
- 訓練データについてのみ、逆伝播により$w$や$b$を`step`メソッドで更新する
- 今回はバッチ学習で全部のデータを入力して誤差を求める

なお、`lossfn = nn.CrossEntropyLoss()`として\
`loss_train = lossfn(y_train, t_train)`などとする例の方が一般的である


In [ ]:
from torch import optim
# SGDを利用
optimizer = optim.SGD(net.parameters(), lr=0.01)  # 学習率は0.01
# 損失のログ
record_loss_train = []
record_loss_test = []
# 1000エポック学習
for i in range(1000):
  # 最初に勾配を初期化つまり0にする
  optimizer.zero_grad()   
  # 順伝播
  y_train = net(x_train)
  y_test = net(x_test)   
  # 交差エントロピー誤差で誤差を求める
  loss_train = nn.CrossEntropyLoss()(y_train, t_train)
  loss_test = nn.CrossEntropyLoss()(y_test, t_test)
  record_loss_train.append(loss_train.item()) # ゼロ次元テンソルからpythonの値を得るには.itemが便利
  record_loss_test.append(loss_test.item())
  # 逆伝播(勾配を求める)
  loss_train.backward()
  # パラメータの更新
  optimizer.step()
  if i%100 == 0:
    print("Epoch:", i, "Loss_Train:", loss_train.item(), "Loss_Test:", loss_test.item())

訓練データ、テストデータ両方の誤差の推移をグラフで表示する
どんどん誤差が低減しているのがわかるであろう

なお、追加で実行するとさらに削減できるが、削減幅はあまり期待できない

グラフを確認し、訓練データとテストデータの最終的な値がほぼ同じであることを確認する
- 異なる場合は過学習
- 同程度に誤差が大きい場合は学習不足

In [ ]:
plt.plot(range(len(record_loss_train)), record_loss_train, label="Train")
plt.plot(range(len(record_loss_test)), record_loss_test, label="Test")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Error")
plt.show()  # ラベルがあるときは、きちんとplt.show()を呼び出すこと

正答率を求めて、モデルの性能を確認する

ここでは、テストデータを用いる

In [ ]:
y_test = net(x_test)
count = (y_test.argmax(1) == t_test).sum().item()
print("正解率:", str(count/len(y_test)*100) + "%")

訓練済みのモデルを使って、任意のデータを入力し、予測させる

実際に、`img_id`を変更して任意の画像を入力し、モデルが機能していることを確かめよう

- 間違いを探そうとすると確率5%程度なので、20回に1回しか成功しないであろう

In [ ]:
img_id = 17   # ここを好きな数字IDに変更するとよい
x_pred = digit_images[img_id]
image = x_pred.reshape(8, 8)
plt.imshow(image, cmap="Greys_r")
plt.show()
x_pred = torch.tensor(x_pred, dtype=torch.float32)
y_pred = net(x_pred)
print("正解:", labels[img_id], "予測結果:", y_pred.argmax().item())

In [ ]:
y_pred

# 課題2(PyTorch MNIST)

scikit-learnの手書き文字認識データを使ったPyTorchによる認識について改良を施す

レポート条件
- 上記の例の結果を添付する
- 改良し、上記よりも少ないエポック数で、上記よりも良い結果を得るモデルを作成して添付する
- 改良は、下記の点に限定すること
  - 指定箇所以外の変更で性能を向上するとチート行為とみなされる
- 自動採点を行うため#Qから始まる部分は残しておくこと

改良点
- ネットワーク[改良点1]
  - 「ここから」「ここまで」の範囲を修正
  - 層の数や、層のノード数を変更するとよい
- 最適化アルゴリズム[改良点2]
  - 修正は1行とすること、複数行になってはいけない
- エポック数[改良点3]
  - 修正は数字だけにすること

下記コード中に、改良点を施す場所を記載しており、また、上記のオリジナルコードも記載している

[改良点1]、[改良点2]、[改良点3]の部分を変更して条件を満たすこと- 今回は、その他の部分の変更は認めない

なお、運で上記の例の結果が悪く、運で改良した結果が良くなった場合も、一応解答として認めるが、追試で成功することを加点要素とする

提出方法はこれまでと同様とする

これまでの学習内容を踏まえれば、十分解答できるが、内容が多岐にわたるため、課題提出期間を拡大します
- 簡単にすぐにできる課題ですが、じっくりと取り組むこともできます
  - 既学習者は様々な工夫を試みてください
  - 初めて学ぶ人は、一つ一つ、概念的な意味をくみ取りながら進めてください
    - また、気になることが少しでもあれば、その周辺を調べてみてください

In [ ]:
import torch
from sklearn import datasets
from sklearn.model_selection import train_test_split
digits_data = datasets.load_digits()
digit_images = digits_data.data
labels = digits_data.target
x_train, x_test, t_train, t_test = train_test_split(digit_images, labels)
x_train = torch.tensor(x_train, dtype=torch.float32)
t_train = torch.tensor(t_train, dtype=torch.int64) 
x_test = torch.tensor(x_test, dtype=torch.float32)
t_test = torch.tensor(t_test, dtype=torch.int64) 

## モデルの工夫[改良点1]

最初の64と最後の10さえ守れば大丈夫(なはず)

In [ ]:
from torch import nn

net = nn.Sequential(
  # ------- [改良点1] #Q1S --------
  # ここから「ここまで」のコードを変更すること
  nn.Linear(64, 32),  # 8x8の画像なので64からスタートするのは固定
  nn.ReLU(),
  nn.Linear(32, 16),
  nn.ReLU(),
  nn.Linear(16, 10)   #0から9の10種の数字に対応するので最後が10も固定
  # ここまで
  # ------- #Q1E -------
)
print(net)

## 学習における最適化アルゴリズムの工夫[改良点2]

様々な最適化アルゴリズムが準備されている
- 学習した、しないに関わらず好きなモデルを使うと良い
- 次のwebページにマニュアルがあり各種最適化アルゴリズムが指定されている
  - https://pytorch.org/docs/stable/optim.html
  - 英語が嫌だとう人は、Googleの変訳はこちら\
  https://translate.google.co.jp/translate?hl=ja&sl=en&tl=ja&u=https%3A%2F%2Fpytorch.org%2Fdocs%2Fstable%2Foptim.html

何してよいかわからないという人は、ひとまず学んだadamを使ってみてはどうだろう

- マニュアルにはadamだけでも様々存在するようだが、シンプルなadamが準備されている
  - もちろんハイパーパラメータもいろいろあるが、全部デフォルトでやってみるとよい
  - adamの威力を知れるであろう
  - ついでに、AMSGradの指定についても調べておくと良い\
  現時点で一般的に高性能とされるのが、AMSGradであろう
  




## エポック数の削減[改良点3]

[改良点1][改良点2]によりエポック数を削減しても、良い結果を得ることができるであろう

- 特に、[改良点2]の工夫はエポック数削減に大きく寄与し、現状のままでは過学習気味になるであろう

そこで、エポック数を削減し調整すること

In [ ]:
from torch import optim
# 次のなんだか追加されている部分は、この部分のセルを何度実行しても
#「追加で学習」ではなく「最初から学習しなおし」になるように
# パラメータをXavierで初期化している
#   その他、Heの初期化などがあり、それぞれ定義されている
#   詳細は省略するが収束しやすいように工夫して初期化しておく、ということである
#   一般に、シグモイド関数であればXavierの初期化がよいとされ、
#   さらにReLUも考慮したのが、Heの初期化である
#   nn.init.kaiming_uniform_(m.weight)とする
#   いずれもuniformは、normalの分布形状を指定できる
def init_weights(m):
  if type(m) == nn.Linear:
    torch.nn.init.xavier_uniform(m.weight)
    m.bias.data.fill_(0.01)
net.apply(init_weights)
# ここまでが初期化のためのコードで無視してよい
# 皆さんはこのセルだけ実行すればよい
# つまり、先ほどのコードは初期化されないため繰り返し押すと学習が追加で進むことになる
lossfn = nn.CrossEntropyLoss()
optimizer =  optim.SGD(net.parameters(), lr=0.01) #[変更点2] #Q2
record_loss_train = []
record_loss_test = []
for i in range(1000): #[変更点3] #Q3
  optimizer.zero_grad()
  y_train = net(x_train)
  y_test = net(x_test)
  loss_train = lossfn(y_train, t_train)
  loss_test = lossfn(y_test, t_test)
  record_loss_train.append(loss_train.item())
  record_loss_test.append(loss_test.item())
  loss_train.backward()
  optimizer.step()
  if i%100 == 0:
    print("Epoch:", i, "Loss_Train:", loss_train.item(), "Loss_Test:", loss_test.item())
import matplotlib.pyplot as plt
plt.plot(range(len(record_loss_train)), record_loss_train, label="Train")
plt.plot(range(len(record_loss_test)), record_loss_test, label="Test")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Error")
plt.show()
y_test = net(x_test)
count = (y_test.argmax(1) == t_test).sum().item()
print("正解率:", str(count/len(y_test)*100) + "%")